In [3]:
## THIS IS THE CODE PROVIDED IN THE LAB SESSION FROM LECTURER ##
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes
    
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])

def bgr8_to_ssd_input(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

class ObjectDetector(object):
    
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn

    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
    def __call__(self, *inputs):
        return self.execute(*inputs)

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [4]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        self.warning_flag=0
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   

    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
                
            #we only consider the central area of the vision sensor
            depth_image[:190,:]=0
            depth_image[290:,:]=0
            depth_image[:,:160]=0
            depth_image[:,480:]=0
            
            #For object avoidance, we don't consider the distance that are lower than 100mm or bigger than 1000mm
            depth_image[depth_image<100]=0
            depth_image[depth_image>1000]=0
            
            #If all of the values in the depth image is 0, the depth[depth!=0] command will fail
            #we set a specific value here to prevent this failure
            depth_image[0,0]=2000
            
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            if (depth_image[depth_image!=0].min()<400):
                self.warning_flag=1
            else:
                self.warning_flag=0
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
               
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

In [5]:
attack_mode = False # Initial attack_mode is false
love_mode = False # Initial love_mode is false
fear_mode = False # Initial fear_mode is false
curious_mode = False # Initial curious_mode is false

def stop_all_behaviour():
    
    """ This functions is created to make all the behaviours mode inactive.
    """
    global attack_mode, love_mode, fear_mode, curious_mode
    #All the modes are set the false
    
    attack_mode = False
    love_mode = False
    fear_mode = False
    curious_mode = False
    
    robot.stop()#Robot will stop if it is moving

In [6]:
def love_behaviour(matching_detections,image):
    
    """ This function is created to give robot love behaviour.
    """
    
    global love_mode # Function uses global love_mode variable
    
    if love_mode: # Check if the robot is in love mode
        largest = [] # initial largest [area, startx, starty , endx, endy]
        leftarea , rightarea =0,0 # Initial left and right areas are set to zero

        for det in matching_detections: # Loop is created to detech all target objects
            bbox = det['bbox'] # Get detected box
            #Created blue box around the object
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])),
                          (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
            startx = int(width * bbox[0])#Take start x point
            starty = int(height * bbox[1])#Take start y point
            endx   = int(width * bbox[2])#Take end x point
            endy   = int(height * bbox[3])#Take end y point
            area   = abs(endx - startx) * abs(endy - starty)#Calculate the area
            
            #If the area bigger then previous take this one as largest
            if len(largest) == 0 or area > largest[0]: 
                largest = [area, startx, starty, endx, endy] # Make it new largest area.

        try: #To avoid fails try statement is added
            if largest[3] < 320 : # If even end of x is smaller then middle point of the screen.
                leftarea = largest[0] # The left area will be all of the objects area.
                rightarea = 0 # There is going to be nothing in the left side.
            elif largest[1] >320 : # If even start of x is bigger then middle point of the screen.
                rightarea= largest[0] # The right area will be all of the objects area.
                leftarea = 0 # There is going to be nothing in the right side.
            elif largest[3] > 320 and largest[1] < 320: # If object is middle of the screen
                # Calculate how much of the object is in the left area
                leftarea = abs(largest[4] - largest[2]) * abs((width/2)-largest[1])
                # Calculate how much of the object is in the right area
                rightarea = abs(largest[4] - largest[2]) * abs(largest[3]-(width/2))
        except:# If there is any fail
            pass # Pass the statement


        if len(matching_detections)==0 or camera.warning_flag == 1:
            # If there is nothing detected or robot is to close to any object
            robot.stop()# Robot will stop
            
        else:# If there is no condition like defined up there
            
            if rightarea > leftarea *3: # If most of size of the object in the right side.

                
                robot.forward_right(0.7) # Robot will move forward right side with 0.7 speed.
                time.sleep(0.2) # Make this move for 0.2 second

            elif leftarea > rightarea *3: # If most of size of the object in the left side.

    
                robot.forward_left(0.7) # Robot will move forward right side with 0.7 speed.
                time.sleep(0.2) # Make this move for 0.2 second

            else: # If none of the area is bigger than other.

                robot.forward(0.4) # Go forward.

In [7]:

def agressive_behaviour(matching_detections,image):
 
    """ This function is created to give robot aggressive behaviour.
    """

    global attack_mode # Function uses global attack_mode variable
    
    if attack_mode: # Check if the robot is in attack_mode

        largest = [] # initial largest [area, startx, starty , endx, endy]
        leftarea , rightarea =0,0 # Initial left and right areas are set to zero

        for det in matching_detections: # Loop is created to detech all target objects
            bbox = det['bbox'] # Get detected box
            #Created blue box around the object
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])),
                          (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
            
            startx = int(width * bbox[0]) #Take start x point
            starty = int(height * bbox[1]) #Take start y point
            endx   = int(width * bbox[2]) #Take end x point
            endy   = int(height * bbox[3]) #Take end y point
            area   = abs(endx - startx) * abs(endy - starty) #Calculate the area

            #If the area bigger then previous take this one as largest
            if len(largest) == 0 or area > largest[0]:
                largest = [area, startx, starty, endx, endy] # Make it new largest area.

        try: #To avoid fails try statement is added
            if largest[3] < 320 :# If even end of x is smaller then middle point of the screen.
                leftarea = largest[0] # The left area will be all of the objects area.
                rightarea = 0 # There is going to be nothing in the left side.
            elif largest[1] >320 : # If even start of x is bigger then middle point of the screen.
                rightarea= largest[0] # The right area will be all of the objects area.
                leftarea = 0  # There is going to be nothing in the right side.
            elif largest[3] > 320 and largest[1] < 320:# If object is middle of the screen
                # Calculate how much of the object is in the left area
                leftarea = abs(largest[4] - largest[2]) * abs((width/2)-largest[1])
                # Calculate how much of the object is in the right area
                rightarea = abs(largest[4] - largest[2]) * abs(largest[3]-(width/2))
        except:# If there is any fail
            pass # Pass the statement

        # If there is no object which is looking for or any object.
        if len(matching_detections)==0 and camera.warning_flag == 0:
            
            robot.left(0.4)#keep turning to find one.
        # If there is no object which is looking for and a another object in front of robot.    
        elif len(matching_detections)==0: 
            
            robot.stop()# Robot will stop.
            
        else: # If robot see target object
            
            if camera.warning_flag == 1: # If robot is close enough to the object.
                
                time.sleep(0.3)# Keep moving forward to go closer target ebject for 0.3 second
                robot.backward(0.7) # Go back
                time.sleep(0.7)# Wait 0.7 seconds
                robot.stop()# Stop in there
                robot.forward(0.7) # Go forward to the target again
                time.sleep(0.7)#Go for 0.7 seconds
                robot.stop()#Stop in there
                attack_mode = False# Change the attack mode with false to finish behaviour.
                stop_all_behaviour() # Stop all behaviours to have new command.

            elif rightarea > leftarea *3 : # If most of size of the object in the right side.

                robot.forward_right(0.8) # Robot will move forward right side with 0.8 speed.
                time.sleep(0.2)# Make this move for 0.2 second

            elif leftarea > rightarea *3: # If most of size of the object in the left side.

                robot.forward_left(0.8)  # Robot will move forward right side with 0.8 speed.
                time.sleep(0.2)# Make this move for 0.2 second

            else: # If none of the area is bigger than other.

                robot.forward(0.4) # Go forward.


In [8]:
def curious_behaviour(matching_detections,image):
    
    """ This function is created to give robot curious behaviour.
    """    
    
    global curious_mode # Function uses global curios_mode variable
    
    if curious_mode : # Check if the robot is in curious_mode

        largest = [] # initial largest [area, startx, starty , endx, endy]
        leftarea , rightarea =0,0 # Initial left and right areas are set to zero

        for det in matching_detections: # Loop is created to detech all target objects
            bbox = det['bbox']# Get detected box
            #Created blue box around the object
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])),
                          (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
            startx = int(width * bbox[0])#Take start x point
            starty = int(height * bbox[1])#Take start y point
            endx   = int(width * bbox[2])#Take end x point
            endy   = int(height * bbox[3])#Take end y point
            area   = abs(endx - startx) * abs(endy - starty)#Calculate the area

            #If the area bigger then previous take this one as largest
            if len(largest) == 0 or area > largest[0]:
                
                largest = [area, startx, starty, endx, endy]# Make it new largest area.

        try: #To avoid fails try statement is added
            if largest[3] < 320 : # If even end of x is smaller then middle point of the screen.
                leftarea = largest[0] # The left area will be all of the objects area.
                rightarea = 0 # There is going to be nothing in the left side.
            elif largest[1] >320 : # If even start of x is bigger then middle point of the screen.
                rightarea= largest[0] # The right area will be all of the objects area.
                leftarea = 0 # There is going to be nothing in the right side.
            elif largest[3] > 320 and largest[1] < 320: # If object is middle of the screen
                # Calculate how much of the object is in the left area
                leftarea = abs(largest[4] - largest[2]) * abs((width/2)-largest[1])
                # Calculate how much of the object is in the right area
                rightarea = abs(largest[4] - largest[2]) * abs(largest[3]-(width/2))
        except:# If there is any fail
            pass# Pass the statement


        # If there is no object which is looking for or any object.
        if len(matching_detections)==0 and camera.warning_flag == 0:
            
            robot.left(0.4)#keep turning to find one.
        # If there is no object which is looking for and a another object in front of robot.    
        elif len(matching_detections)==0: 
            
            robot.stop()# Robot will stop.
            
        else: # If robot see target object
            
            if camera.warning_flag == 1: # If robot is close enough to the object.
                
                for _ in range(5): # Make angles changer move for five times
                    
                    robot.degree(80)# Turn 80 degree right
                    time.sleep(0.5)# Wait for 0.5 second
                    robot.forward(0.5) # Go forward for half speed
                    time.sleep(0.5)# Wait for 0.5 second
                    robot.degree(-85) # Tun 85 degree left
                    time.sleep(0.5) # Wait for 0.5 second
                    robot.stop() # Stop the robot
                    time.sleep(1) # Wait for 1 second
                    stop_all_behaviour() # Stop all behaviuors
                    

            elif rightarea > leftarea *3 : # If most of size of the object in the right side.

                robot.forward_right(0.8) # Robot will move forward right side with 0.8 speed.
                time.sleep(0.2)# Make this move for 0.2 second

            elif leftarea > rightarea *3: # If most of size of the object in the left side.

                robot.forward_left(0.8)  # Robot will move forward right side with 0.8 speed.
                time.sleep(0.2)# Make this move for 0.2 second

            else: # If none of the area is bigger than other.

                robot.forward(0.4) # Go forward.


In [9]:
def fear_behaviour(matching_detections,image):
    
    """ This function is created to give robot fear behaviour.
    """       
    
    global fear_mode # Function uses global curios_mode variable
    
    if fear_mode: # Check if the robot is in fear_mode

        # If there is no object what we fear stay there
        if len(matching_detections)==0:
            
            robot.stop()# Robot will stop.

        else:# If robot see object that it afraid of

            robot.left(1) # Turn back
            time.sleep(0.8)# Wait for 0.8 second
            robot.forward(1)# Go forward for one second.
            time.sleep(0.5)# Wait for 0.5 second


In [10]:
import ipywidgets.widgets as widgets # ipywidgets are imported
from IPython.display import display, HTML # display and HTML are imported
from RobotClass import Robot # Robot library is imported
import math # Math library is imported 
import time # TIme library is imported 

width = 640 # Camera screen width is set to 640
height = 480 # Camera screen height is set to 480

image_widget = widgets.Image(format='jpeg', width=300, height=300)# Initial image widget is set
label_widget = widgets.IntText(value=1, description='tracked label')# Initial label is set to 1 which is human

robot = Robot()# Robot class is created


def processing(change):

    """ This function is created to observes changes to have new dynamic values.
    """
    image = change['new'] # Captures changes in the image 
    imgsized= cv2.resize(image,(300,300)) # Resize the image into 300x300
    # compute all detected objects
    
    detections = model(imgsized)# Stores the detections
    # Stores if there is any detections which matches with label value
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    fear_behaviour(matching_detections, image)# Fear behaviour is applied
    curious_behaviour(matching_detections, image)# Curious behaviour is applied
    love_behaviour(matching_detections,image)# Love behaviour is applied
    agressive_behaviour(matching_detections,image)# Agressive behaviour is applied
    image_widget.value = bgr8_to_jpeg(image)# Take the image value
    
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
camera.observe(processing, names='color_value')

In [11]:
# Create robot behaviour selection and control buttons
button_aggressive = widgets.Button(description='Aggressive')
button_love = widgets.Button(description='Love')
button_fear = widgets.Button(description='Fear')
button_curious = widgets.Button(description='Curious')
button_stop_behaviour = widgets.Button(description='Stop Behaviour', button_style='danger')
button_up = widgets.Button(description='↑')
button_down = widgets.Button(description='↓')
button_left = widgets.Button(description='←')
button_right = widgets.Button(description='→')
button_forward_left = widgets.Button(description='↖')
button_forward_right = widgets.Button(description='↗')
button_stop = widgets.Button(description='Stop', button_style='danger') 

# styling buttons
button_up.layout.width = 'auto'
button_down.layout.width = 'auto'
button_left.layout.width = 'auto'
button_right.layout.width = 'auto'
button_stop.layout.width = 'auto'
button_forward_left.layout.width = 'auto'
button_forward_right.layout.width = 'auto'

#defining button functions

def on_aggressive_click(b):

    """ This function is created to make attack_mode is True when it is clicked.  
    """
    
    global attack_mode, love_mode, fear_mode, curious_mode
    attack_mode = True
    love_mode = False
    fear_mode = False
    curious_mode = False
    
def on_love_click(b):
    
    """ This function is created to make love_mode is True when it is clicked.  
    """    
    
    global attack_mode, love_mode, fear_mode, curious_mode
    attack_mode = False
    love_mode = True
    fear_mode = False
    curious_mode = False


def on_fear_click(b):
    
    """ This function is created to make fear_mode is True when it is clicked.  
    """      
    
    global attack_mode, love_mode, fear_mode, curious_mode
    attack_mode = False
    love_mode = False
    fear_mode = True
    curious_mode = False

def on_curious_click(b):

    """ This function is created to make curious_mode is True when it is clicked.  
    """  
    
    global attack_mode, love_mode, fear_mode, curious_mode
    attack_mode = False
    love_mode = False
    fear_mode = False
    curious_mode = True
    
def on_stop_behaviour_click(b):
    
    """ This function is created to stop all behaviours when it is clicked.  
    """  
    
    stop_all_behaviour()

    
def on_forward_left_click(b):
    
    """ This function is created to move robot forward left when it is clicked.  
    """  
    
    robot.forward_left(0.5)
    time.sleep(0.5)
    robot.stop()
    stop_all_behaviour()
    
def on_forward_click(b):
    
    """ This function is created to move robot forward when it is clicked.  
    """     
    
    robot.forward(0.5)
    time.sleep(0.5)
    robot.stop()
    stop_all_behaviour()
    
    
def on_forward_right_click(b):
    
    """ This function is created to move robot forward right when it is clicked.  
    """ 
    
    robot.forward_right(0.5)
    time.sleep(0.5)
    robot.stop()
    stop_all_behaviour()
    
def on_left_click(b):
    
    """ This function is created to move robot left when it is clicked.  
    """     
    
    robot.left(0.5)
    time.sleep(0.5)
    robot.stop()
    stop_all_behaviour()

def on_stop_click(b):
    robot.stop()
    stop_all_behaviour()

def on_right_click(b):
    
    """ This function is created to move robot right when it is clicked.  
    """       
    
    robot.right(0.5)
    time.sleep(0.5)
    robot.stop()
    stop_all_behaviour()

def on_backward_click(b):
    
    """ This function is created to move robot backward when it is clicked.  
    """       
    
    robot.backward(0.5)
    time.sleep(0.5)
    robot.stop()   
    stop_all_behaviour()

# Link button clicks to their respective functions
button_aggressive.on_click(on_aggressive_click)
button_love.on_click(on_love_click)
button_fear.on_click(on_fear_click)
button_curious.on_click(on_curious_click)
button_stop_behaviour.on_click(on_stop_behaviour_click)

button_forward_left.on_click(on_forward_left_click)
button_up.on_click(on_forward_click)
button_forward_right.on_click(on_forward_right_click)
button_left.on_click(on_left_click)
button_stop.on_click(on_stop_click)
button_right.on_click(on_right_click)
button_down.on_click(on_backward_click)

# Putting buttons together to have better apperance
button_up_fl_fr_box = widgets.HBox([button_forward_left, button_up, button_forward_right], layout=widgets.Layout(justify_content='center'))
button_lr_stop_box = widgets.HBox([button_left, button_stop, button_right], layout=widgets.Layout(justify_content='center'))
joystick_box = widgets.VBox([button_up_fl_fr_box, button_lr_stop_box, button_down], layout=widgets.Layout(align_items='center'))

# Display buttons
buttons_container = widgets.HBox([button_aggressive, button_love, button_fear, button_curious, button_stop_behaviour])
display(buttons_container)
display(widgets.VBox([widgets.HBox([image_widget,]),label_widget]))
display(joystick_box)
